In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import  StandardScaler
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models.resnet import resnet18
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import torch.utils.data as data
from tqdm import tqdm
from PIL import Image

In [3]:
# set device
if torch.cuda.is_available():
    print('Using GPU')
    device = torch.device('cuda')
else:
    print('Using CPU')
    device = torch.device('cpu')

Using GPU


In [4]:
# Settings
setting = {
    'batch_size': 256,
    'epochs': 2,
    'lr': 0.001,
    'feature_dim': 128,
}

In [5]:
class SimModel(nn.Module):
    def __init__(self, feature_dim):
        super(SimModel, self).__init__()

        self.encoder = []
        for name, module in resnet18().named_children():
            if name == 'conv1':
                module = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
            if not isinstance(module, nn.Linear) and not isinstance(module, nn.MaxPool2d):
                self.encoder.append(module)
        # encoder
        self.encoder = nn.Sequential(*self.encoder)
        # projection head
        self.g = nn.Sequential(
            nn.Linear(512, 512, bias=False),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Linear(512, feature_dim, bias=True)
        )

    def forward(self, x):
        x = self.encoder(x)
        feature = torch.flatten(x, start_dim=1)
        out = self.g(feature)
        return F.normalize(feature, dim=-1), F.normalize(out, dim=-1)

In [6]:
# DataLoader
train_set = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_set = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_loader = data.DataLoader(train_set, batch_size=setting['batch_size'], shuffle=False)
knn_train_loader = data.DataLoader(train_set, batch_size=5000, shuffle=False)

test_loader = data.DataLoader(test_set, batch_size=setting['batch_size'], shuffle=False)
knn_test_loader = data.DataLoader(test_set, batch_size=5000, shuffle=False)


class CIFAR10Pair(datasets.CIFAR10):
    def __getitem__(self, index):
        img, target = self.data[index], self.targets[index]
        img = Image.fromarray(img)

        if self.transform is not None:
            pos_1 = self.transform(img)
            pos_2 = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return pos_1, pos_2, target

 
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    # transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])


train_data = CIFAR10Pair(root='data', train=True, transform=train_transform, download=True)
augmeted_loader = data.DataLoader(train_data, batch_size=setting['batch_size'], shuffle=True, pin_memory=True,drop_last=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [7]:
model = SimModel(setting['feature_dim'])
model.to(device)

# Optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=setting['lr'],weight_decay=1e-6)

In [12]:
def test_data():

    train_imgs, train_labs = next(iter(knn_train_loader))
    test_imgs, test_labs = next(iter(knn_test_loader))
    model.to(device)
    model.eval()
    
    train_codes, _ = model(train_imgs.to(device))
    print('latent layer', train_codes.shape)
    test_codes, _ = model(test_imgs.to(device))
    
    test_codes = test_codes.detach().cpu().numpy()
    train_codes = train_codes.detach().cpu().numpy()
    
    KNN(train_codes.flatten(1), train_labs,
        test_codes.flatten(1), test_labs)

def KNN(train_data, train_labs, test_data, test_labs):
    # Log time

    KNN = KNeighborsClassifier(n_neighbors=200)

    scaler = StandardScaler()
    train_data = scaler.fit_transform(train_data)
    test_data = scaler.fit_transform(test_data)

    KNN.fit(train_data, train_labs)
    predicted = KNN.predict(test_data)

    # In order to pretty print output
    print('KNN Accuracy', accuracy_score(predicted, test_labs)*100, '%')
    
    kneighbors = KNN.kneighbors(test_data[0], return_distance=False)
    print('KNN Neighbors', kneighbors)
    print(len(kneighbors))
    


In [9]:
# Train

temperature = 0.5
def train():
    for epoch in range(setting['epochs']):
        total_loss, total_num, train_bar = 0.0, 0, tqdm(augmeted_loader)
        for pics1, pics2, labels in train_bar:
            pics1 = pics1.to(device)
            pics2 = pics2.to(device)

            # Forward
            codes, out_1 = model(pics1)
            codes, out_2 = model(pics2)
            
            out_1 = torch.flatten(out_1, start_dim=1)
            out_2 = torch.flatten(out_2, start_dim=1)

            out = torch.cat([out_1, out_2], dim=0)
            # [2*B, 2*B]
            sim_matrix = torch.exp(torch.mm(out, out.t().contiguous()) / temperature)
            mask = (torch.ones_like(sim_matrix) - torch.eye(2 * setting['batch_size'], device=sim_matrix.device)).bool()
            # [2*B, 2*B-1]
            sim_matrix = sim_matrix.masked_select(mask).view(2 * setting['batch_size'], -1)

            # compute loss
            pos_sim = torch.exp(torch.sum(out_1 * out_2, dim=-1) / temperature)
            # [2*B]
            pos_sim = torch.cat([pos_sim, pos_sim], dim=0)
            loss = (- torch.log(pos_sim / sim_matrix.sum(dim=-1))).mean()
            
            # Backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_num += setting['batch_size']
            total_loss += loss.item() * setting['batch_size']
            train_bar.set_description('Train Epoch: [{}/{}] Loss: {:.4f}'.format(epoch + 1, setting['epochs'], total_loss / total_num))

In [10]:
print('with: ',device)
print('Start training...')
train()
model.to('cpu')
with torch.no_grad():
    torch.cuda.empty_cache()

with:  cuda
Start training...


Train Epoch: [2/2] Loss: 4.6430: 100%|██████████| 195/195 [01:36<00:00,  2.02it/s]


In [13]:

test_data()

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.22 GiB (GPU 0; 8.00 GiB total capacity; 6.41 GiB already allocated; 0 bytes free; 6.43 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF